In [1]:
# import the MNIST dataset
import cv2
import numpy as np
from keras.models import load_model
from collections import deque

In [2]:
# Load the model
model = load_model('best_model.h5')

In [3]:
# Define letters in terms of decimal numbers as dictionary
letters = {i: chr(i+97) for i in range(26)}
letters[26] = ''

In [4]:
# Define the marker color in hsv format
blue_lower = np.array([100,60,60])
blue_upper = np.array([140,255,255])

In [5]:
# Define a new kernel
kernel = np.ones((5,5), np.uint8)

In [6]:
# Define a blackboard to write the letters
black_board = np.zeros((480,640,3), dtype=np.uint8)
alphabete = np.zeros((200,200,3), dtype=np.uint8)

In [7]:
# Define dequens used for storing alphabet drawn on screen 
points = deque(maxlen=512)

In [11]:
# Intalize the camera using opencv to capture
cap = cv2.VideoCapture(0)
prediction = 26     # Intialy the prediction display empty
while True:
    ret, frame=cap.read()

    # Convert the frame to hsv and gray
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
    
    blue = cv2.inRange(hsv,blue_lower, blue_upper)
    # Erosion
    blue = cv2.erode(blue, kernel)
    # opening
    blue = cv2.morphologyEx(blue, cv2.MORPH_OPEN, kernel)
    # Dilution
    blue = cv2.dilate(blue, kernel)

    # Find countors 
    conts, _ = cv2.findContours(blue, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    center = None

    # if countors find
    if len(conts)>0:
        cont = sorted(conts, key=cv2.contourArea, reverse=True)[0]
        ((x,y), raduis)=cv2.minEnclosingCircle(cont)
        cv2.circle(frame, (int(x), int(y),), int(raduis), (125,344,278), 2)
        M = cv2.moments(cont)
        center = (int(M['m10']/M['m00']), int(M['m01']/M['m00'])) 
        points.appendleft(center)
    elif len(conts) == 0:
        if len(points) != 0:
            black_board_gray = cv2.cvtColor(black_board, cv2.COLOR_BGR2GRAY) 
            blur = cv2.medianBlur(black_board_gray, 15)
            blur = cv2.GaussianBlur(blur, (5,5), 0)
            thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]

            black_board_conts = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[0]
            if len(black_board_conts)>0:
                cont = sorted(black_board_conts, key= cv2.contourArea, reverse=True)[0]
                if cv2.contourArea(cont)>1000:
                    x,y,w,h = cv2.boundingRect(cont)
                    alphabete = black_board_gray[y-10:y+h+10, x-10:x+w+10]
                    try:
                        img = cv2.resize(alphabete, (28,28))
                    except cv2.error as e:
                        continue
                    img = np.array(img)
                    img = img.astype('float32')/255.0
                    prediction = model.predict(img.reshape(1,28,28))[0]

                    prediction = np.argmax(prediction) # prediction in form of percentage
            
            # Clear the deque point and the blackborad 
            points = deque(maxlen=512)
            black_board = np.zeros((480,640,3), dtype=np.uint8)
    # Pass the detected points with lines 
    for i in range(1, len(points)):
        if points[i-1] is None or points[i] is None:
            continue
        cv2.line(frame, points[i-1], points[i], (0,0,0), 2)
        cv2.line(black_board, points[i-1], points[i], (255,255,255), 6)
    # Get the dimensions of the frame
    height, width, _ = frame.shape
    # Define the position of the text
    x = 350-int(width / 2) # Top-Left aligned 
    y = 30  # Top of the window
    cv2.putText(frame, "Predicted Alphabete: " + str(letters[int(prediction)]), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)


    cv2.imshow('Alphabete recognition', frame)
    if cv2.waitKey(1) & 0xFF == ord('q') or cv2.waitKey(1) == 27:   # Press q for exiting from the webcam
        break
cap.release()
cv2.destroyAllWindows()